# Recommendation

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD
import pickle



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "E:\pycharm general venv\GeneralVenv\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "E:\pycharm general venv\GeneralVenv\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "E:\pycharm general venv\GeneralVenv\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.sta

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

## Load Processed Data

In [2]:
file_path = "processed_data/movielens_processed.csv"
data = pd.read_csv(file_path)

ratings = data[['user_id', 'item_id', 'rating']]
ratings_matrix = ratings.pivot_table(index="user_id", columns="item_id", values="rating")


## Load or Recompute Models

In [6]:
# User similarity
user_sim_matrix = cosine_similarity(ratings_matrix.fillna(0))
user_sim_matrix = pd.DataFrame(user_sim_matrix, index=ratings_matrix.index, columns=ratings_matrix.index)

# Item similarity
item_sim_matrix = cosine_similarity(ratings_matrix.fillna(0).T)
item_sim_matrix = pd.DataFrame(item_sim_matrix, index=ratings_matrix.columns, columns=ratings_matrix.columns)






## Recommendation Functions

In [4]:
def recommend_user_cf(user_id, user_sim_matrix, ratings_matrix, n=5):
    sims = user_sim_matrix.loc[user_id].drop(user_id)
    weighted_ratings = sims @ ratings_matrix.fillna(0).values
    preds = pd.Series(weighted_ratings, index=ratings_matrix.columns)
    rated_items = ratings_matrix.loc[user_id].dropna().index
    preds = preds.drop(rated_items, errors="ignore")
    return preds.sort_values(ascending=False).head(n)

def recommend_item_cf(user_id, item_sim_matrix, ratings_matrix, n=5):
    user_ratings = ratings_matrix.loc[user_id].dropna()
    scores = item_sim_matrix.loc[user_ratings.index].T @ user_ratings
    scores = scores / item_sim_matrix.loc[user_ratings.index].sum()
    scores = scores.drop(user_ratings.index, errors="ignore")
    return scores.sort_values(ascending=False).head(n)

def recommend_svd(user_id, svd, ratings_matrix, n=5):
    user_ratings = ratings_matrix.loc[user_id]
    predictions = {
        item: svd.predict(user_id, item).est
        for item in ratings_matrix.columns
        if pd.isna(user_ratings.get(item, np.nan))
    }
    return sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:n]


## Example Usage

In [7]:
target_user_id = 13
k_movies = 5

print(f"\nTop {k_movies} recommendations for user {target_user_id}:\n")

print("=== User-Based CF ===")
print(recommend_user_cf(target_user_id, user_sim_matrix, ratings_matrix, n=k_movies))

print("\n=== Item-Based CF ===")
print(recommend_item_cf(target_user_id, item_sim_matrix, ratings_matrix, n=k_movies))




Top 5 recommendations for user 13:

=== User-Based CF ===


Exception: Dot product shape mismatch, (942,) vs (943, 1682)